In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

In [3]:
from google.colab import drive

In [4]:
# get size of the variable in megabytes
def get_variable_size(var):
    display(sys.getsizeof(var)/ (1024**2))

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
taxi_data_clean = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/taxi_data_clean_3.csv')

In [7]:
print('Shape of data: {}'.format(taxi_data_clean.shape))
print('Columns: {}'.format(taxi_data_clean.columns))

Shape of data: (1458233, 28)
Columns: Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_date', 'pickup_hour', 'pickup_day_of_week',
       'pickup_holiday', 'total_distance', 'total_travel_time',
       'number_of_steps', 'haversine_distance', 'direction', 'geo_cluster',
       'temperature', 'visibility', 'wind speed', 'precip', 'events',
       'average_speed', 'trip_duration_log'],
      dtype='object')


In [8]:
train_data = taxi_data_clean.copy()
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,haversine_distance,direction,geo_cluster,temperature,visibility,wind speed,precip,events,average_speed,trip_duration_log
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,1.498521,99.970196,9,4.4,8.0,27.8,0.3,None,15.896176,6.122493
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,1.805507,-117.153768,4,28.9,16.1,7.4,0.0,None,13.646335,6.498282
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,6.385098,-159.680165,4,-6.7,16.1,24.1,0.0,None,18.747119,7.661527
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,1.485498,-172.737700,4,7.2,16.1,25.9,0.0,None,14.932028,6.063785
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,1.188588,179.473585,9,9.4,16.1,9.3,0.0,None,13.364690,6.077642


### Задание 4.1.
Сразу позаботимся об очевидных неинформативных и избыточных признаках.

а) Какой из признаков является уникальным для каждой поездки и не несет полезной информации в определении ее продолжительности?

б) Утечка данных (data leak) - это…

в) Подумайте, наличие какого из признаков в обучающем наборе данных создает утечку данных?

г) Исключите выбранные в пунктах а) и в) признаки из исходной таблицы с данными. Сколько столбцов в таблице у вас осталось?


In [9]:
columns_to_drop = ['id', 'dropoff_datetime']
train_data = train_data.drop(columns_to_drop, axis=1)

In [10]:
train_data.shape

(1458233, 26)

Ранее мы извлекли всю необходимую для нас информацию из даты начала поездки, теперь мы можем избавиться от этих признаков, так как они нам больше не понадобятся:


In [11]:
drop_columns = ['pickup_datetime', 'pickup_date']
train_data = train_data.drop(drop_columns, axis=1)
print('Shape of data: {}'.format(train_data.shape))

Shape of data: (1458233, 24)


### Задание 4.2.
Закодируйте признак vendor_id в таблице train_data таким образом, чтобы он был равен 1, если идентификатор таксопарка равен 0, и 1 в противном случае.

Закодируйте признак store_and_fwd_flag в таблице train_data таким образом, чтобы он был равен 0, если флаг выставлен в значение "N", и 0 в противном случае.

а) Рассчитайте среднее по закодированному столбцу vendor_id. Ответ приведите с точностью до сотых.

б) Рассчитайте среднее по закодированному столбцу store_and_fwd_flag. Ответ приведите с точностью до тысячных.



In [12]:
train_data['vendor_id'] = train_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)

In [13]:
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].apply(lambda x: 0 if x == "N" else 1)

In [14]:
unique_vendor_id = train_data['vendor_id'].unique()
unique_store_fwd_flag = train_data['store_and_fwd_flag'].unique()

In [15]:
display(unique_vendor_id)

array([1, 0])

In [16]:
display(unique_store_fwd_flag)

array([0, 1])

In [17]:
train_data['vendor_id'].mean()

0.53497486341346

In [18]:
train_data['store_and_fwd_flag'].mean()

0.005514207948935458

### Задание 4.3.
Создайте таблицу data_onehot из закодированных однократным кодированием признаков pickup_day_of_week, geo_cluster и events в таблице train_data с помощью OneHotEndoder из библиотеки sklearn. Параметр drop выставите в значение 'first', чтобы удалять первый бинарный столбец, тем самым не создавая излишних признаков.

В результате работы OneHotEncoder вы получите безымянный numpy-массив, который нам будет необходимо преобразовать обратно в DataFrame, для более удобной работы в дальнейшем. Чтобы получить имена закодированных столбцов у объекта типа OneHotEncoder есть специальный метод get_feature_names_out(). Он возвращает список новых закодированных имен столбцов в формате <оригинальное имя столбца>_<имя категории>.

Пример использования:

``` python
# Получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()
# Составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns=column_names)
```

В этом псевдокоде:
* one_hot_encoder - объект класса OneHotEncoder
* data_onehot - numpy-массив, полученный в результате трансформации кодировщиком

В результате выполнения задания у вас должен быть образован DataFrame `data_onehot`, который содержит кодированные категориальные признаки pickup_day_of_week, geo_cluster и events.


Сколько бинарных столбцов у вас получилось сгенерировать с помощью однократного кодирования?


In [19]:
data_onehot = preprocessing.OneHotEncoder(drop='first', handle_unknown='ignore')

columns = ['pickup_day_of_week', 'geo_cluster', 'events']

df_onehot = train_data[columns]

data_onehot.fit(df_onehot)

encoded_columns = data_onehot.get_feature_names_out()
display(len(encoded_columns))

data_onehot = data_onehot.transform(df_onehot)
data_onehot = data_onehot.toarray()
onehot_df = pd.DataFrame(data_onehot, columns=encoded_columns)

18

In [20]:
onehot_df.head()

,pickup_day_of_week_1,pickup_day_of_week_2,pickup_day_of_week_3,pickup_day_of_week_4,pickup_day_of_week_5,pickup_day_of_week_6,geo_cluster_1,geo_cluster_2,geo_cluster_3,geo_cluster_4,geo_cluster_5,geo_cluster_6,geo_cluster_7,geo_cluster_8,geo_cluster_9,events_None,events_Rain,events_Snow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


Добавим полученную таблицу с закодированными признаками:

In [21]:
train_data = pd.concat(
    [train_data.reset_index(drop=True).drop(columns, axis=1), onehot_df],
    axis=1
)
print('Shape of data: {}'.format(train_data.shape))

Shape of data: (1458233, 39)


Теперь, когда категориальные признаки предобработаны, сформируем матрицу наблюдений X, вектор целевой переменной y и его логарифм y_log. В матрицу наблюдений войдут все столбцы из таблицы с поездками за исключением целевого признака trip_duration и его логарифмированной версии trip_duration_log:


In [22]:
X = train_data.drop(['trip_duration', 'trip_duration_log', 'visibility', 'wind speed', 'precip'], axis=1)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

In [23]:
train_data.isna().any()

vendor_id               False
passenger_count         False
pickup_longitude        False
pickup_latitude         False
dropoff_longitude       False
dropoff_latitude        False
store_and_fwd_flag      False
trip_duration           False
pickup_hour             False
pickup_holiday          False
total_distance          False
total_travel_time       False
number_of_steps         False
haversine_distance      False
direction               False
temperature             False
visibility              False
wind speed              False
precip                  False
average_speed           False
trip_duration_log       False
pickup_day_of_week_1    False
pickup_day_of_week_2    False
pickup_day_of_week_3    False
pickup_day_of_week_4    False
pickup_day_of_week_5    False
pickup_day_of_week_6    False
geo_cluster_1           False
geo_cluster_2           False
geo_cluster_3           False
geo_cluster_4           False
geo_cluster_5           False
geo_cluster_6           False
geo_cluste

In [24]:
display(train_data['total_distance'].isna().sum())
display(train_data['total_travel_time'].isna().sum())
display(train_data['number_of_steps'].isna().sum())
display(train_data['temperature'].isna().sum())
display(train_data['visibility'].isna().sum())
display(train_data['wind speed'].isna().sum())
display(train_data['average_speed'].isna().sum())

0

0

0

0

0

0

0

Все наши модели мы будем обучать на логарифмированной версии y_log.

Выбранный тип валидации - hold-out. Разобьем выборку на обучающую и валидационную в соотношении 67/33:

In [25]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X, y_log,
    test_size=0.33,
    random_state=42
)

На данный момент у нас достаточно много признаков: скорее всего, не все из них будут важны. Давайте оставим лишь те, которые сильнее всего связаны с целевой переменной и точно будут вносить вклад в повышение качества модели.


### Задание 4.4.
С помощью SelectKBest отберите 25 признаков, наилучшим образом подходящих для предсказания целевой переменной в логарифмическом масштабе. Отбор реализуйте по обучающей выборке, используя параметр score_func = f_regression.

Укажите признаки, которые вошли в список отобранных


In [26]:
f_regression = feature_selection.f_regression
choice_of_fitch = feature_selection.SelectKBest(score_func=f_regression, k=25)
choice_of_fitch_fit = choice_of_fitch.fit(X_train, y_train_log)
final_columns = choice_of_fitch_fit.get_feature_names_out()

In [27]:
print(final_columns)
print(len(final_columns))

['vendor_id' 'passenger_count' 'pickup_longitude' 'pickup_latitude'
 'dropoff_longitude' 'dropoff_latitude' 'store_and_fwd_flag' 'pickup_hour'
 'pickup_holiday' 'total_distance' 'total_travel_time' 'number_of_steps'
 'haversine_distance' 'temperature' 'average_speed' 'pickup_day_of_week_1'
 'pickup_day_of_week_2' 'pickup_day_of_week_3' 'pickup_day_of_week_4'
 'pickup_day_of_week_5' 'pickup_day_of_week_6' 'geo_cluster_3'
 'geo_cluster_5' 'geo_cluster_7' 'geo_cluster_9']
25


In [28]:
X_train = X_train[final_columns]
X_valid = X_valid[final_columns]

Так как мы будем использовать различные модели, в том числе внутри которых заложены численные методы оптимизации, то давайте заранее позаботимся о масштабировании факторов.


### Задание 4.5.
Нормализуйте предикторы в обучающей и валидационной выборках с помощью MinMaxScaler из библиотеки sklearn. Помните, что обучение нормализатора производится на обучающей выборке, а трансформация на обучающей и валидационной!

Рассчитайте среднее арифметическое для первого предиктора (т. е. для первого столбца матрицы) из валидационной выборки. Ответ округлите до сотых.


In [29]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(X_train)

X_train = pd.DataFrame(min_max_scaler.transform(X_train), columns=final_columns)
X_valid = pd.DataFrame(min_max_scaler.transform(X_valid), columns=final_columns)

In [30]:
round(np.mean(X_valid.iloc[:, 0]), 2)

0.54

In [31]:
X_train.to_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/X_train_4.csv')

In [32]:
X_valid.to_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/X_valid_4.csv')

In [33]:
y_train_log.to_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/y_train_log_4.csv')

In [34]:
y_valid_log.to_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/y_valid_log_4.csv')